# Open-domain Question answering

## Retriever

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 27.1 MB/s 
     |████████████████████████████████| 1.1 MB 61.3 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
     |████████████████████████████████| 212 kB 61.5 MB/s 
     |████████████████████████████████| 140 kB 65.3 MB/s 
     |████████████████████████████████| 596 kB 66.2 MB/s 
     |████████████████████████████████| 127 kB 75.5 MB/s 
     |████████████████████████████████| 271 kB 74.3 MB/s 
     |████████████████████████████████| 144 kB 67.5 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's de

### load dataset for train

In [2]:
from datasets import load_dataset

sq = load_dataset('squad', split='train')
sq

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [3]:
sq[0]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [4]:
sq.shape

(87599, 5)

In [5]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 4.4 MB 55.1 MB/s 
     |████████████████████████████████| 1.2 MB 59.9 MB/s 
     |████████████████████████████████| 6.6 MB 55.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=d9b6577d5c49a5301e2d37a85e7e211013a5ec36c2eaf795616f6a3584afb15b
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [6]:
from sentence_transformers import InputExample
train = []
for data in sq:
    train.append(InputExample(texts=[data['question'], data['context']]))

In [7]:
import numpy as np
np.shape(train)

(87599,)

In [8]:
from sentence_transformers import datasets
batch_size = 16
loader = datasets.NoDuplicatesDataLoader(train, batch_size=batch_size)

### initialize the model

In [9]:
from sentence_transformers import models, SentenceTransformer

bert = models.Transformer('microsoft/mpnet-base')
pool = models.Pooling(bert.get_word_embedding_dimension(),pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[bert, pool])

model

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/mpnet-base were not used when initializing MPNetModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing MPNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461k [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

### train

In [10]:
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(model)

In [11]:
epochs = 1
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(train_objectives=[(loader, loss)],epochs=epochs,warmup_steps=warmup_steps,output_path='mpnet-mnr-squad',show_progress_bar=True)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5474 [00:00<?, ?it/s]

### load dataset for validation

In [12]:
sq_val = load_dataset('squad', split='validation')

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [13]:
sq_val[0]

{'answers': {'answer_start': [177, 177, 177],
  'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'id': '56be4db0acb8001400a502ec',
 'question': 'Which NFL team represented the AFC at Super Bo

In [14]:
sq_val.shape

(10570, 5)

### preparing data for evaluation

In [15]:
import pandas as pd

data_val = pd.DataFrame()
for data in sq_val:
    data_val = data_val.append({'question': data['question'],'context': data['context'],'id': data['id']}, ignore_index=True)

In [16]:
data_val.head(5)

,question,context,id
0,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502ec
1,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502ed
2,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502ee
3,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502ef
4,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502f0


In [17]:
remove_dupe = data_val.drop_duplicates(subset='context',keep='first')
remove_dupe = remove_dupe.drop(columns=['question'])
remove_dupe['id'] = remove_dupe['id'] + 'context'

In [18]:
remove_dupe.head(5)


,context,id
0,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502eccontext
30,The Panthers finished the regular season with ...,56be4e1facb8001400a502f6context
55,The Broncos took an early lead in Super Bowl 5...,56be4eafacb8001400a50302context
80,"CBS broadcast Super Bowl 50 in the U.S., and c...",56be5333acb8001400a5030acontext
106,"In early 2012, NFL Commissioner Roger Goodell ...",56be53b8acb8001400a50314context


In [19]:
data_val = data_val.merge(remove_dupe, how='inner', on='context')
data_val.head(5)

,question,context,id_x,id_y
0,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502ec,56be4db0acb8001400a502eccontext
1,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502ed,56be4db0acb8001400a502eccontext
2,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502ee,56be4db0acb8001400a502eccontext
3,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502ef,56be4db0acb8001400a502eccontext
4,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,56be4db0acb8001400a502f0,56be4db0acb8001400a502eccontext


In [ ]:
dict_qu = {row['id_x']: row['question'] for i, row in data_val.iterrows()}

In [ ]:
dict_con = {row['id_y']: row['context'] for i, row in data_val.iterrows()}

In [ ]:
relevant_docs = {key: [] for key in data_val['id_x'].unique()}
for i, row in data_val.iterrows():
    relevant_docs[row['id_x']].append(row['id_y'])
relevant_docs = {key: set(values) for key, values in relevant_docs.items()}

### our retriever evaluation

In [23]:
from sentence_transformers import evaluation
IReval = evaluation.InformationRetrievalEvaluator(dict_qu, dict_con, relevant_docs,
                                                  accuracy_at_k=[20,100],map_at_k=[20,100],
                                                  mrr_at_k=[20,100],ndcg_at_k=[20,100],
                                                  precision_recall_at_k=[20,100],show_progress_bar=True)

In [ ]:
IReval.compute_metrices(model)

Batches:   0%|          | 0/331 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:31<00:00, 31.99s/it]


{'cos_sim': {'accuracy@k': {20: 0.9656575212866604, 100: 0.9957426679280984},
  'map@k': {20: 0.7295302301604603, 100: 0.7304380642764586},
  'mrr@k': {20: 0.729530230160452, 100: 0.7304380642764511},
  'ndcg@k': {20: 0.7856256893297394, 100: 0.7914685011497196},
  'precision@k': {20: 0.048282876064333034, 100: 0.009957426679280982},
  'recall@k': {20: 0.9656575212866604, 100: 0.9957426679280984}},
 'dot_score': {'accuracy@k': {20: 0.9577105014191107, 100: 0.9954588457899716},
  'map@k': {20: 0.6889744928992858, 100: 0.6901168118236197},
  'mrr@k': {20: 0.6889744928992775, 100: 0.6901168118236121},
  'ndcg@k': {20: 0.7523365553948699, 100: 0.7596762225170691},
  'precision@k': {20: 0.04788552507095554, 100: 0.009954588457899717},
  'recall@k': {20: 0.9577105014191107, 100: 0.9954588457899716}}}

### single dpr evaluation

In [ ]:
query_encoder_single = SentenceTransformer('facebook-dpr-question_encoder-single-nq-base')
passage_encoder_single = SentenceTransformer('facebook-dpr-ctx_encoder-single-nq-base')
IReval.compute_metrices(query_encoder_single,passage_encoder_single)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/331 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:32<00:00, 32.08s/it]


{'cos_sim': {'accuracy@k': {20: 0.8458845789971617, 100: 0.9514664143803216},
  'map@k': {20: 0.497647143705886, 100: 0.5005706607155151},
  'mrr@k': {20: 0.49764714370588087, 100: 0.5005706607155104},
  'ndcg@k': {20: 0.5774082362592238, 100: 0.5974220455640514},
  'precision@k': {20: 0.04229422894985809, 100: 0.009514664143803217},
  'recall@k': {20: 0.8458845789971617, 100: 0.9514664143803216}},
 'dot_score': {'accuracy@k': {20: 0.8790917691579944, 100: 0.9630085146641438},
  'map@k': {20: 0.5476500437788994, 100: 0.5500467175346739},
  'mrr@k': {20: 0.5476500437788939, 100: 0.5500467175346689},
  'ndcg@k': {20: 0.6243051803680614, 100: 0.6403597413931532},
  'precision@k': {20: 0.04395458845789972, 100: 0.009630085146641439},
  'recall@k': {20: 0.8790917691579944, 100: 0.9630085146641438}}}

### multi dpr evaluation

In [ ]:
query_encoder_multi = SentenceTransformer('facebook-dpr-question_encoder-multiset-base')
passage_encoder_multi = SentenceTransformer('facebook-dpr-ctx_encoder-multiset-base')
IReval.compute_metrices(query_encoder_multi,passage_encoder_multi)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/673 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/331 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:12<00:00, 12.59s/it]


{'cos_sim': {'accuracy@k': {20: 0.8542100283822138, 100: 0.9536423841059603},
  'map@k': {20: 0.5380812224910357, 100: 0.5408514650666483},
  'mrr@k': {20: 0.5380812224910303, 100: 0.5408514650666435},
  'ndcg@k': {20: 0.611239611064166, 100: 0.6301251783950603},
  'precision@k': {20: 0.04271050141911069, 100: 0.009536423841059604},
  'recall@k': {20: 0.8542100283822138, 100: 0.9536423841059603}},
 'dot_score': {'accuracy@k': {20: 0.8752128666035951, 100: 0.9650898770104068},
  'map@k': {20: 0.569169699258456, 100: 0.5716706235100327},
  'mrr@k': {20: 0.56916969925845, 100: 0.5716706235100277},
  'ndcg@k': {20: 0.6400865004994306, 100: 0.6571390996155118},
  'precision@k': {20: 0.04376064333017975, 100: 0.009650898770104069},
  'recall@k': {20: 0.8752128666035951, 100: 0.9650898770104068}}}

### encode contexts

In [24]:
unique_contexts = []
unique_ids = []
for d in sq_val:
    if d['context'] not in unique_contexts:
        unique_contexts.append(d['context'])
        unique_ids.append(d['id'])
sq_val = sq_val.filter(lambda x: True if x['id'] in unique_ids else False)
sq_val

Parameter 'function'=<function <lambda> at 0x7ff65f50d8c0> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/11 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2067
})

In [25]:
sq_val = sq_val.map(lambda x: {'encoding': model.encode(x['context']).tolist()}, batched=True, batch_size=4)
sq_val

  0%|          | 0/517 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'encoding'],
    num_rows: 2067
})

### storing the vectors

In [26]:
!pip install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 160 kB 31.8 MB/s 
     |████████████████████████████████| 58 kB 7.1 MB/s 
     |████████████████████████████████| 269 kB 70.4 MB/s 


In [27]:
import pinecone

In [28]:
API_KEY = '3d6d64f9-dc38-4f28-9c12-851517875827'

pinecone.init(api_key=API_KEY, environment='us-west1-gcp')

In [29]:
model.get_sentence_embedding_dimension()


768

In [30]:
if 'squad-index' not in pinecone.list_indexes():
    pinecone.create_index(name='squad-index', dimension=model.get_sentence_embedding_dimension(), metric='cosine')

In [31]:
index = pinecone.Index('squad-index')

In [32]:
upserts = [(v['id'], v['encoding'], {'text': v['context']}) for v in sq_val]
for i in range(0, len(upserts), 50):
    i_end = i + 50
    if i_end > len(upserts): i_end = len(upserts)
    index.upsert(vectors=upserts[i:i_end])

### examples of our retriver

In [ ]:
q1=data_val['question'][4]
q1

'What color was used to emphasize the 50th anniversary of the Super Bowl?'

In [ ]:
c1=data_val['context'][4]
c1

'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'

In [ ]:
q1_encode = model.encode([q1]).tolist()

In [ ]:
top_c1 = index.query(q1_encode, top_k=3, include_metadata=True)
top_c1

{'matches': [{'id': '56be4db0acb8001400a502ec',
              'metadata': {'text': 'Super Bowl 50 was an American football '
                                   'game to determine the champion of the '
                                   'National Football League (NFL) for the '
                                   '2015 season. The American Football '
                                   'Conference (AFC) champion Denver Broncos '
                                   'defeated the National Football Conference '
                                   '(NFC) champion Carolina Panthers 24–10 to '
                                   'earn their third Super Bowl title. The '
                                   'game was played on February 7, 2016, at '
                                   "Levi's Stadium in the San Francisco Bay "
                                   'Area at Santa Clara, California. As this '
                                   'was the 50th Super Bowl, the league '
                        

In [64]:
q2=data_val['question'][10000]
q2

'Which country today is a remnant of the Ottoman empire?'

In [65]:
c2=data_val['context'][10000]
c2

'With Istanbul as its capital and control of lands around the Mediterranean basin, the Ottoman Empire was at the center of interactions between the Eastern and Western worlds for six centuries. Following a long period of military setbacks against European powers, the Ottoman Empire gradually declined into the late nineteenth century. The empire allied with Germany in the early 20th century, with the imperial ambition of recovering its lost territories, but it dissolved in the aftermath of World War I, leading to the emergence of the new state of Turkey in the Ottoman Anatolian heartland, as well as the creation of modern Balkan and Middle Eastern states, thus ending Turkish colonial ambitions.'

In [ ]:
q2_encode = model.encode([q2]).tolist()

In [ ]:
top_c2 = index.query(q2_encode, top_k=2, include_metadata=True)
top_c2

{'matches': [{'id': '5730bd00069b5314008322d7',
              'metadata': {'text': 'With Istanbul as its capital and control '
                                   'of lands around the Mediterranean basin, '
                                   'the Ottoman Empire was at the center of '
                                   'interactions between the Eastern and '
                                   'Western worlds for six centuries. '
                                   'Following a long period of military '
                                   'setbacks against European powers, the '
                                   'Ottoman Empire gradually declined into the '
                                   'late nineteenth century. The empire allied '
                                   'with Germany in the early 20th century, '
                                   'with the imperial ambition of recovering '
                                   'its lost territories, but it dissolved in '
                    

## Reader

### load dataset for train

In [33]:
sq_reader = load_dataset('squad', split='train')
sq_reader

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [34]:
sq_reader[0]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [35]:
sq_reader[0]['answers']['answer_start'][0] + len(sq_reader[0]['answers']['text'][0])

541

In [36]:
sq_reader[0]['context'][515:541]

'Saint Bernadette Soubirous'

### function for add end of answer

In [37]:
def answer_end(sample):
  token_end=sample['answers']['answer_start'][0] + len(sample['answers']['text'][0])
  return {'answers':{'answer_start': [sample['answers']['answer_start'][0]], 'text': [sample['answers']['text'][0]],'answer_end': [token_end]}}

In [38]:
sq_reader = sq_reader.map(lambda x: answer_end(x))

  0%|          | 0/87599 [00:00<?, ?ex/s]

In [39]:
sq_reader[0]

{'answers': {'answer_end': [541],
  'answer_start': [515],
  'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

### initialize a tokenizer

In [40]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [41]:
sq_reader = sq_reader.map(lambda x: tokenizer(
    x['question'], x['context'], max_length=300,
    padding='max_length', truncation=True,
    return_offsets_mapping=True
))

  0%|          | 0/87599 [00:00<?, ?ex/s]

In [ ]:
sq_reader[0]

{'answers': {'answer_end': [541],
  'answer_start': [515],
  'text': ['Saint Bernadette Soubirous']},
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [ ]:
tokenizer.decode(sq_reader[0]['input_ids'])

'[CLS] to whom did the virgin mary allegedly appear in 1858 in lourdes france? [SEP] architecturally, the school has a catholic character. atop the main building\'s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ), is a simple, modern stone statue of mary. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

### length of question and context

In [42]:
def qc_len(sample):
  question_len = 0
  for x in sample['token_type_ids']:
      if x != 1:
          question_len += 1
      else: break
  context_len = sum(sample['token_type_ids'])
  return question_len, context_len

In [ ]:
qc_len(sq_reader[0])

(17, 159)

In [ ]:
sq_reader[0]['answers']['answer_end'][0]

541

### find start position and end position of answer

In [43]:
def char_to_id(sample):
  char_start = sample['answers']['answer_start'][0]
  char_end = sample['answers']['answer_end'][0]
  question_len, context_len=qc_len(sample)
  context_mappings = sample['offset_mapping'][question_len:][:context_len-1]
  for i, mapping in enumerate(context_mappings):
      if char_start >= mapping[0] and char_start <= mapping[1]:
          token_start = question_len + i
      if char_end >= mapping[0] and char_end <= mapping[1]:
          token_end = question_len + i
          return {'start_positions': token_start, 'end_positions': token_end}
      if i == len(context_mappings) - 1:
          token_start, token_end = 0, 0
          return {'start_positions': token_start, 'end_positions': token_end}

In [44]:
sq_reader = sq_reader.map(lambda x: char_to_id(x))

  0%|          | 0/87599 [00:00<?, ?ex/s]

In [ ]:
print(sq_reader[0]['answers'])
print(sq_reader[0]['start_positions'])
print(sq_reader[0]['end_positions'])

{'answer_end': [541], 'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}
130
137


In [ ]:
tokenizer.decode(sq_reader[0]['input_ids'][130:138])

'saint bernadette soubirous'

In [ ]:
print(sq_reader[12]['answers'])
print(sq_reader[12]['start_positions'])
print(sq_reader[12]['end_positions'])

{'answer_end': [245], 'answer_start': [234], 'text': ['Old College']}
59
60


In [ ]:
tokenizer.decode(sq_reader[12]['input_ids'][59:61])

'old college'

In [45]:
sq_reader= sq_reader.remove_columns(['id', 'title', 'context', 'question', 'answers', 'offset_mapping'])
sq_reader

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 87599
})

### prepare format of data being fed into model

In [46]:
from transformers import default_data_collator
data_collator = default_data_collator

### initialize the model

In [47]:
from transformers import BertForQuestionAnswering

model_reader = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

### train

In [50]:
from transformers import TrainingArguments

args = TrainingArguments(
    'bert-base-uncased-squad',
    learning_rate=2e-5,per_device_train_batch_size=12,num_train_epochs=1,weight_decay=0.1,warmup_steps=int(len(sq_reader)*epochs*0.1)
    )

In [51]:
from transformers import Trainer
import torch

trainer = Trainer(model_reader,args,train_dataset=sq_reader,data_collator=data_collator,tokenizer=tokenizer)

In [52]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 87599
  Num Epochs = 1
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 7300


Step,Training Loss
500,5.477900
1000,4.277700
1500,2.856800
2000,2.404600
2500,2.080700
3000,1.752900
3500,1.545300
4000,1.456800
4500,1.381900
5000,1.299100


Saving model checkpoint to bert-base-uncased-squad/checkpoint-500
Configuration saved in bert-base-uncased-squad/checkpoint-500/config.json
Model weights saved in bert-base-uncased-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-base-uncased-squad/checkpoint-500/special_tokens_map.json
Saving model checkpoint to bert-base-uncased-squad/checkpoint-1000
Configuration saved in bert-base-uncased-squad/checkpoint-1000/config.json
Model weights saved in bert-base-uncased-squad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-squad/checkpoint-1000/tokenizer_config.json
Special tokens file saved in bert-base-uncased-squad/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to bert-base-uncased-squad/checkpoint-1500
Configuration saved in bert-base-uncased-squad/checkpoint-1500/config.json
Model weights saved in bert-base-uncased-squa

TrainOutput(global_step=7300, training_loss=2.064067683807791, metrics={'train_runtime': 5369.934, 'train_samples_per_second': 16.313, 'train_steps_per_second': 1.359, 'total_flos': 1.34117194882716e+16, 'train_loss': 2.064067683807791, 'epoch': 1.0})

In [53]:
trainer.save_model('bert-reader-squad')

Saving model checkpoint to bert-reader-squad
Configuration saved in bert-reader-squad/config.json
Model weights saved in bert-reader-squad/pytorch_model.bin
tokenizer config file saved in bert-reader-squad/tokenizer_config.json
Special tokens file saved in bert-reader-squad/special_tokens_map.json


### pipeline for reader 

In [54]:
from transformers import pipeline

model_name = 'bert-reader-squad'
reader = pipeline('question-answering',model=model_name,tokenizer=model_name)

loading configuration file bert-reader-squad/config.json
Model config BertConfig {
  "_name_or_path": "bert-reader-squad",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file bert-reader-squad/config.json
Model config BertConfig {
  "_name_or_path": "bert-reader-squad",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": nu

In [55]:
sq_val

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'encoding'],
    num_rows: 2067
})

In [56]:
answers = sq_val['answers']
sq_val= sq_val.remove_columns(['id', 'title', 'answers','encoding'])

### example of our reader

In [57]:
sq_val[4]

{'context': 'In early 2012, NFL Commissioner Roger Goodell stated that the league planned to make the 50th Super Bowl "spectacular" and that it would be "an important game for us as a league".',
 'question': 'Who was the NFL Commissioner in early 2012?'}

In [58]:
answers[4]

{'answer_start': [32, 32, 38],
 'text': ['Roger Goodell', 'Roger Goodell', 'Goodell']}

In [59]:
reader({
    'question': sq_val[4]['question'],
    'context': sq_val[4]['context']
})

{'answer': 'Roger Goodell',
 'end': 45,
 'score': 0.9961608648300171,
 'start': 32}

## function for open-domain QA

In [60]:
def OD_QA(q):
  a=[]
  q_encode = model.encode([q]).tolist()
  top_k=3
  top_c = index.query(q_encode, top_k=top_k, include_metadata=True)
  for i in range(top_k):
    context=top_c['matches'][i]['metadata']['text']
    a.append(reader({'question': q,'context': context}))
  return a

### examples

In [62]:
a=OD_QA(sq_val[4]['question'])
a

[{'answer': 'Roger Goodell',
  'end': 45,
  'score': 0.9961608648300171,
  'start': 32},
 {'answer': 'Gary Kubiak',
  'end': 302,
  'score': 0.8788304924964905,
  'start': 291},
 {'answer': 'Rivera', 'end': 11, 'score': 0.1644708663225174, 'start': 5}]

In [66]:
q2

'Which country today is a remnant of the Ottoman empire?'

In [67]:
c2

'With Istanbul as its capital and control of lands around the Mediterranean basin, the Ottoman Empire was at the center of interactions between the Eastern and Western worlds for six centuries. Following a long period of military setbacks against European powers, the Ottoman Empire gradually declined into the late nineteenth century. The empire allied with Germany in the early 20th century, with the imperial ambition of recovering its lost territories, but it dissolved in the aftermath of World War I, leading to the emergence of the new state of Turkey in the Ottoman Anatolian heartland, as well as the creation of modern Balkan and Middle Eastern states, thus ending Turkish colonial ambitions.'

In [68]:
OD_QA(q2)

[{'answer': 'Turkey', 'end': 557, 'score': 0.5796698927879333, 'start': 551},
 {'answer': 'Europe', 'end': 299, 'score': 0.047510527074337006, 'start': 293},
 {'answer': '1924', 'end': 332, 'score': 0.4107491672039032, 'start': 328}]

## true answers and predicted answers by our model

In [69]:
from tqdm import tqdm

In [70]:
len(sq_val)

2067

In [71]:
results = []
for i in tqdm(range(len(sq_val))):
  a=OD_QA(sq_val[i]['question'])
  pred=[]
  for j in range(len(a)):
    pred.append(a[j]['answer'])
  results.append({
        'predicted_answers':pred ,
        'true_answer': answers[i]['text']
  })

100%|██████████| 2067/2067 [1:13:38<00:00,  2.14s/it]


In [72]:
results[0]

{'predicted_answers': ['Denver Broncos',
  'NFL Commissioner Roger Goodell',
  "Levi's Stadium"],
 'true_answer': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']}

## evaluate our model with exact match

In [73]:
sum=0
for j in range(len(results)):
  k=len(results[j]['true_answer'])
  for d in range(k):
    exact=int(results[j]['predicted_answers'][0]==results[j]['true_answer'][d] or results[j]['predicted_answers'][1]==results[j]['true_answer'][d] or results[j]['predicted_answers'][2]==results[j]['true_answer'][d])
    if exact==1 :
      sum+=1
      break

In [74]:
sum

1336

In [75]:
exact_match=sum/len(results)
exact_match

0.6463473633284954